# **Introduction**

## **Chi-test (boroughs + svi)**
## **Bar-chart with svi as regression/scatterplot (boroughs first)**

source: https://www.atsdr.cdc.gov/place-health/media/pdfs/2024/10/SVI2022Documentation.pdf

source: https://www.atsdr.cdc.gov/place-health/php/svi/svi-interactive-map.html

Chi-test: with the whole dataset considered, the chi-test gives much more moderate and accurate result. There is no statistically strong association between high svi nta and high evi nta. (But if you move into a nta with higher-than-average svi, you are still 2.8 times more likely to get evicted compared to living in a lower than average svi nta). There is a statistically strong association between high black + hispanics nta with high evi nta. (and the odds are 3.81).

In [2]:
# !pip install geopandas folium matplotlib seaborn scipy
# !pip install esda
# !pip install splot
# !pip install geopandas contextily
# # for google colab, had to reinstall some pacakges.

In [ ]:
# !pip install geopandas folium matplotlib seaborn scipy esda splot

In [3]:
import pandas as pd
import geopandas as gpd
import numpy as np
import datetime as dt
import scipy

from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from shapely.geometry import Point
from sklearn.neighbors import NearestNeighbors

# visualization
import matplotlib.pyplot as plt
from matplotlib import colors as mcolors
import seaborn as sns
import folium
from folium.plugins import HeatMap
from folium import Marker
from folium.plugins import MarkerCluster
import plotly.express as px
import plotly.io as pio


import contextily as ctx
from scipy.stats import f_oneway
from sklearn.decomposition import PCA
from functools import reduce
from scipy.stats import chi2_contingency
import statsmodels.api as sm

# spatial statistics
from esda.moran import Moran
from esda.getisord import G_Local
from libpysal.weights import Queen, Rook

# system and utility
import warnings
import os
import io
from IPython.display import IFrame
from google.colab import files

from libpysal.weights import Queen, Rook
from esda.moran import Moran
import matplotlib.pyplot as plt
from splot.esda import moran_scatterplot

# suppress warnings
warnings.filterwarnings('ignore')

# inline
%matplotlib inline

In [4]:
pd.set_option('display.float_format', lambda x: '%.4f' % x)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# **Step 1 Get the Eviction data**

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# data source:
file_path1 = '/content/drive/My Drive/X999/bbl_evictions_311_svi_normal_times.csv'
file_path2 = '/content/drive/My Drive/X999/bbl_evictions_311_svi_covid.csv'

In [7]:
evictions_pre_post_raw = pd.read_csv(file_path1)
evictions_covid_raw = pd.read_csv(file_path2)
evictions_covid_raw.shape, evictions_pre_post_raw.shape
# 91 and 92 with normal time + one more analysis column regarding an svi item

((5386, 91), (66397, 92))

In [8]:
evictions_pre_post = evictions_pre_post_raw.copy()
evictions_covid = evictions_covid_raw.copy()

In [9]:
evictions_pre_post.head(2)

,primary_key,bbl,court_index_number,docket_number,eviction_address,eviction_apartment_number,executed_date,borough,zipcode,ejectment,eviction/legal_possession,latitude,longitude,community_board,council_district,census_tract,bin,nta,year,month_year,geometry,average_year_eviction_count,yearbuilt,bldgclass,numfloors,unitsres,ownername,bldgarea,building_type,building_category,is_condo,floor_category,rent_era,architectural_style,economic_period,residential_units_category,is_llc,building_size_category,size_quartile,decade,fips,e_totpop,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_aian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white,invalid_zip,svi_quartile,svi_group,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints
0,*308072/22_5865,3037420029,*308072/22,5865,356 MILLER AVE,1 AND BASEMENT,2024-12-04,BROOKLYN,11207,Not an Ejectment,Possession,40.6721,-73.8911,5.0000,37.0000,1152.0000,3083989,East New York,2024,2024-12,POINT (-73.891105 40.672121),0.8000,1930.0000,C0,3.0000,3.0000,356 MILLER LLC,2700.0000,pre-war,walk-up,False,low-rise,"Pre-1947, pre-rent-control","1921–1930, Art Deco Skyscrapers","1930-1945, great depression and WWII",3-5 units,True,small,Q3 (50-75%),1930-1939,11207,96801.0000,0.9788,0.9140,0.9808,0.9812,0.9839,33.9000,11.1000,19.1000,6.0000,13.8000,22.5000,13.8000,5.3000,57.8000,9.1000,44.7000,55.9000,32.8000,1.5000,0.0000,0.0000,2.9000,1.6000,94.7000,5.3000,False,Q3,medium-high,0.0000,0.0000,1.0000,0.0000,1.0000,2.0000,0.0000,0.0000,1.0000,0.0000,3.0000,0.0000,0.0000,0.0000,0.0000,3.0000,0.0000,0.0000,0.0000,3.0000,5.0000,19.0000
1,*313639/23_5202,3057940012,*313639/23,5202,710 61ST STREET,2ND FLOOR,2024-03-04,BROOKLYN,11220,Not an Ejectment,Possession,40.6359,-74.0119,7.0000,38.0000,118.0000,3143881,Sunset Park East,2024,2024-03,POINT (-74.011883 40.635941),0.6000,1920.0000,B2,2.0000,2.0000,"A.R.M. PARKING, LLC",1204.0000,pre-war,two-family,False,low-rise,"Pre-1947, pre-rent-control","1900–1920, Beaux-Arts","Pre-1929, pre-great depression",2-unit,True,very small,Q1 (smallest 25%),1920-1929,11220,93008.0000,0.9885,0.7635,0.9594,0.9179,0.9662,37.5000,7.5000,37.9000,11.6000,13.1000,25.4000,8.4000,40.2000,61.7000,23.7000,43.6000,1.7000,40.9000,40.7000,0.4000,0.0000,1.2000,0.2000,85.0000,15.0000,False,Q3,medium-high,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,1.0000,1.0000,4.0000


In [10]:
evictions_covid.head(2)

,primary_key,bbl,court_index_number,docket_number,eviction_address,eviction_apartment_number,executed_date,borough,zipcode,ejectment,eviction/legal_possession,latitude,longitude,community_board,council_district,census_tract,bin,nta,year,month_year,geometry,average_year_eviction_count,yearbuilt,bldgclass,numfloors,unitsres,ownername,bldgarea,building_type,building_category,is_condo,floor_category,rent_era,architectural_style,economic_period,residential_units_category,is_llc,building_size_category,size_quartile,decade,fips,e_totpop,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_aian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white,invalid_zip,svi_quartile,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints
0,004123/20_209969,2032140141,004123/20,209969,2541 A GRAND AVE,ROOM 3B,2022-08-22,BRONX,10468,Not an Ejectment,Possession,40.8654,-73.9013,7.0000,14.0000,265.0000,2113173,Kingsbridge Heights,2022,2022-08,POINT (-73.901317 40.865396),0.2000,2004.0000,C0,3.0000,3.0000,MONJU SARKER,3420.0000,post-war,walk-up,False,low-rise,"1994–Present, vacancy decontrol","2001-present, New Architecture","1991–2008, modern economic growth",3-5 units,False,medium-small,Q4 (largest 25%),2000-2009,10468,81397.0000,0.9954,0.9407,0.9870,0.9470,0.9874,39.5000,11.6000,28.3000,9.2000,11.2000,26.4000,12.2000,26.9000,71.8000,19.2000,56.7000,15.6000,78.0000,2.3000,0.0000,0.0000,0.5000,0.5000,96.9000,3.1000,False,Q3,0.0000,0.0000,0.0000,0.0000,3.0000,0.0000,0.0000,2.0000,0.0000,0.0000,1.0000,0.0000,2.0000,0.0000,0.0000,2.0000,0.0000,0.0000,0.0000,3.0000,1.0000,14.0000
1,0050153/20_106030,4031560133,0050153/20,106030,98-05 67TH AVENUE,12F,2022-04-14,QUEENS,11375,Not an Ejectment,Possession,40.7242,-73.8556,6.0000,29.0000,71306.0000,4074666,Forest Hills,2022,2022-04,POINT (-73.855552 40.724241),0.2000,1960.0000,D3,13.0000,181.0000,MARSEILLES LEASING LIMITED PARTNERSHIP,177710.0000,post-war,elevator,False,high-rise,"1947–1969, rent-control","1951–1980, the International Style, Alternativ...","1946–1975, pst war economic boom",100+ units,False,mega,Q4 (largest 25%),1960-1969,11375,75212.0000,0.4759,0.5698,0.8789,0.8057,0.7322,12.0000,4.8000,6.1000,3.7000,20.4000,18.0000,10.5000,7.9000,41.9000,5.8000,25.4000,2.7000,16.4000,28.5000,0.1000,0.0000,4.6000,0.7000,53.0000,47.0000,False,Q1 (Low),0.0000,2.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,2.0000,0.0000,62.0000,0.0000,34.0000,0.0000,0.0000,4.0000,1.0000,0.0000,0.0000,2.0000,5.0000,112.0000


In [11]:
evictions_pre_post.columns, \
evictions_covid.columns, \
evictions_pre_post.shape, \
evictions_covid.shape

(Index(['primary_key', 'bbl', 'court_index_number', 'docket_number',
        'eviction_address', 'eviction_apartment_number', 'executed_date',
        'borough', 'zipcode', 'ejectment', 'eviction/legal_possession',
        'latitude', 'longitude', 'community_board', 'council_district',
        'census_tract', 'bin', 'nta', 'year', 'month_year', 'geometry',
        'average_year_eviction_count', 'yearbuilt', 'bldgclass', 'numfloors',
        'unitsres', 'ownername', 'bldgarea', 'building_type',
        'building_category', 'is_condo', 'floor_category', 'rent_era',
        'architectural_style', 'economic_period', 'residential_units_category',
        'is_llc', 'building_size_category', 'size_quartile', 'decade', 'fips',
        'e_totpop', 'rpl_theme1', 'rpl_theme2', 'rpl_theme3', 'rpl_theme4',
        'rpl_themes', 'ep_pov150', 'ep_unemp', 'ep_nohsdp', 'ep_uninsur',
        'ep_age65', 'ep_age17', 'ep_disabl', 'ep_limeng', 'ep_noveh',
        'ep_crowd', 'ep_hburd', 'ep_afam', 'ep_hisp

In [12]:
link = '/content/drive/My Drive/X999/svi_cleaned.csv'

In [13]:
svi_df = pd.read_csv(link)
svi_df.head(2)

,fips,location,area_sqmi,e_totpop,m_totpop,e_hu,m_hu,e_hh,m_hh,e_pov150,m_pov150,e_unemp,m_unemp,e_hburd,m_hburd,e_nohsdp,m_nohsdp,e_uninsur,m_uninsur,e_age65,m_age65,e_age17,m_age17,e_disabl,m_disabl,e_sngpnt,m_sngpnt,e_limeng,m_limeng,e_minrty,m_minrty,e_munit,m_munit,e_mobile,m_mobile,e_crowd,m_crowd,e_noveh,m_noveh,e_groupq,m_groupq,ep_pov150,mp_pov150,ep_unemp,mp_unemp,ep_hburd,mp_hburd,ep_nohsdp,mp_nohsdp,ep_uninsur,mp_uninsur,ep_age65,mp_age65,ep_age17,mp_age17,ep_disabl,mp_disabl,ep_sngpnt,mp_sngpnt,ep_limeng,mp_limeng,ep_minrty,mp_minrty,ep_munit,mp_munit,ep_mobile,mp_mobile,ep_crowd,mp_crowd,ep_noveh,mp_noveh,ep_groupq,mp_groupq,epl_pov150,epl_unemp,epl_hburd,epl_nohsdp,epl_uninsur,spl_theme1,rpl_theme1,epl_age65,epl_age17,epl_disabl,epl_sngpnt,epl_limeng,spl_theme2,rpl_theme2,epl_minrty,spl_theme3,rpl_theme3,epl_munit,epl_mobile,epl_crowd,epl_noveh,epl_groupq,spl_theme4,rpl_theme4,spl_themes,rpl_themes,f_pov150,f_unemp,f_hburd,f_nohsdp,f_uninsur,f_theme1,f_age65,f_age17,f_disabl,f_sngpnt,f_limeng,f_theme2,f_minrty,f_theme3,f_munit,f_mobile,f_crowd,f_noveh,f_groupq,f_theme4,f_total,e_daypop,e_noint,m_noint,e_afam,m_afam,e_hisp,m_hisp,e_asian,m_asian,e_aian,m_aian,e_nhpi,m_nhpi,e_twomore,m_twomore,e_otherrace,m_otherrace,ep_noint,mp_noint,ep_afam,mp_afam,ep_hisp,mp_hisp,ep_asian,mp_asian,ep_aian,mp_aian,ep_nhpi,mp_nhpi,ep_twomore,mp_twomore,ep_otherrace,mp_otherrace
0,10001,ZCTA5 10001,0.6238,27004,1827,16975,831,14375,782,5248,797,761,266,3314,531,1930,534,831,289,3428,432,2694,643,2310,499,501,215,1381,405,13460,2305,15840,898,15,23,389,135,12285,840,2213,218,20.3000,2.7000,4.3000,1.5000,23.1000,3.5000,9.1000,2.4000,3.1000,1.0000,12.7000,1.6000,10.0000,2.1000,8.6000,1.9000,3.5000,1.5000,5.3000,1.5000,49.8000,7.8000,93.3000,2.7000,0.1000,0.1000,2.7000,0.9000,85.5000,2.8000,8.2000,0.6000,0.6108,0.4574,0.5573,0.5902,0.4436,2.6593,0.5688,0.1420,0.1161,0.1891,0.4707,0.8777,1.7956,0.1692,0.8670,0.8670,0.8670,0.9853,0.2710,0.7402,0.9949,0.9104,3.9018,0.9806,9.2237,0.7414,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,3,3,239407,1047,389,2220,576,5206,943,5031,774,0,25,0,25,780,326,223,169,7.3000,2.6000,8.2000,2.2000,19.3000,3.0000,18.6000,2.9000,0.0000,0.1000,0.0000,0.1000,2.9000,1.2000,0.8000,0.6000
1,10002,ZCTA5 10002,0.8223,76518,2894,39094,1241,36028,1326,27908,2853,2833,574,14688,1367,18301,1376,4074,766,17681,1287,10028,1549,9896,1062,2211,499,18393,1640,56964,3226,35725,1677,16,28,2461,449,29828,1403,2090,39,36.8000,3.5000,7.6000,1.4000,40.8000,3.5000,30.0000,2.0000,5.4000,1.0000,23.1000,1.7000,13.1000,1.8000,13.0000,1.4000,6.1000,1.4000,24.7000,2.0000,74.4000,3.1000,91.4000,3.2000,0.0000,0.1000,6.8000,1.2000,82.8000,1.8000,2.7000,0.1000,0.9148,0.7946,0.9219,0.9741,0.7207,4.3261,0.9639,0.7296,0.1831,0.5186,0.7390,0.9944,3.1647,0.8781,0.9369,0.9369,0.9369,0.9790,0.0000,0.9105,0.9915,0.7730,3.6540,0.9254,12.0817,0.9656,1,0,1,1,0,3,0,0,0,0,1,1,1,1,1,0,1,1,0,3,8,64307,8590,1110,6141,1194,19864,2190,28477,1989,74,83,24,45,1810,486,574,394,23.8000,2.9000,8.0000,1.5000,26.0000,2.5000,37.2000,2.2000,0.1000,0.1000,0.0000,0.1000,2.4000,0.6000,0.8000,0.5000


In [14]:
svi_df.shape

(204, 153)

In [15]:
# list(svi_df.columns)

In [16]:
svi_df.ep_nhpi.unique()

array([ 0.00e+00,  1.00e-01,  3.00e-01,  2.00e-01, -9.99e+02,  8.00e-01,
        1.20e+00,  5.00e-01,  4.00e-01])

# **Step 2: SVI items**

q quick double check

In [17]:
link = "/content/drive/My Drive/X999/NewYork_ZCTA.csv"

In [18]:
svi_raw = pd.read_csv(link)
svi_raw.head(2)

,ST,STATE,ST_ABBR,FIPS,LOCATION,AREA_SQMI,E_TOTPOP,M_TOTPOP,E_HU,M_HU,E_HH,M_HH,E_POV150,M_POV150,E_UNEMP,M_UNEMP,E_HBURD,M_HBURD,E_NOHSDP,M_NOHSDP,E_UNINSUR,M_UNINSUR,E_AGE65,M_AGE65,E_AGE17,M_AGE17,E_DISABL,M_DISABL,E_SNGPNT,M_SNGPNT,E_LIMENG,M_LIMENG,E_MINRTY,M_MINRTY,E_MUNIT,M_MUNIT,E_MOBILE,M_MOBILE,E_CROWD,M_CROWD,E_NOVEH,M_NOVEH,E_GROUPQ,M_GROUPQ,EP_POV150,MP_POV150,EP_UNEMP,MP_UNEMP,EP_HBURD,MP_HBURD,EP_NOHSDP,MP_NOHSDP,EP_UNINSUR,MP_UNINSUR,EP_AGE65,MP_AGE65,EP_AGE17,MP_AGE17,EP_DISABL,MP_DISABL,EP_SNGPNT,MP_SNGPNT,EP_LIMENG,MP_LIMENG,EP_MINRTY,MP_MINRTY,EP_MUNIT,MP_MUNIT,EP_MOBILE,MP_MOBILE,EP_CROWD,MP_CROWD,EP_NOVEH,MP_NOVEH,EP_GROUPQ,MP_GROUPQ,EPL_POV150,EPL_UNEMP,EPL_HBURD,EPL_NOHSDP,EPL_UNINSUR,SPL_THEME1,RPL_THEME1,EPL_AGE65,EPL_AGE17,EPL_DISABL,EPL_SNGPNT,EPL_LIMENG,SPL_THEME2,RPL_THEME2,EPL_MINRTY,SPL_THEME3,RPL_THEME3,EPL_MUNIT,EPL_MOBILE,EPL_CROWD,EPL_NOVEH,EPL_GROUPQ,SPL_THEME4,RPL_THEME4,SPL_THEMES,RPL_THEMES,F_POV150,F_UNEMP,F_HBURD,F_NOHSDP,F_UNINSUR,F_THEME1,F_AGE65,F_AGE17,F_DISABL,F_SNGPNT,F_LIMENG,F_THEME2,F_MINRTY,F_THEME3,F_MUNIT,F_MOBILE,F_CROWD,F_NOVEH,F_GROUPQ,F_THEME4,F_TOTAL,E_DAYPOP,E_NOINT,M_NOINT,E_AFAM,M_AFAM,E_HISP,M_HISP,E_ASIAN,M_ASIAN,E_AIAN,M_AIAN,E_NHPI,M_NHPI,E_TWOMORE,M_TWOMORE,E_OTHERRACE,M_OTHERRACE,EP_NOINT,MP_NOINT,EP_AFAM,MP_AFAM,EP_HISP,MP_HISP,EP_ASIAN,MP_ASIAN,EP_AIAN,MP_AIAN,EP_NHPI,MP_NHPI,EP_TWOMORE,MP_TWOMORE,EP_OTHERRACE,MP_OTHERRACE
0,36,New York,NY,6390,ZCTA5 06390,4.0467,53,39,253,49,19,19,17,16,0,13,9,26,0,13,27,34,0,13,6,11,31,33,0,18,9,53,20,51,0,18,4,5,0,18,0,13,17,16,32.1000,18.8000,0.0000,52.7000,47.4000,100.0000,0.0000,51.4000,50.9000,45.7000,0.0000,45.2000,11.3000,19.0000,58.5000,41.4000,0.0000,94.7000,17.0000,99.2000,37.7000,92.1000,0.0000,7.1000,1.6000,2.0000,0.0000,94.7000,0.0000,75.5000,32.1000,18.8000,0.8790,0.0000,0.9635,0.0000,0.9960,2.8385,0.6342,0.0000,0.1408,0.9944,0.0000,0.9775,2.1127,0.3009,0.8062,0.8062,0.8062,0.0000,0.4654,0.0000,0.0000,0.9735,1.4389,0.2205,7.1963,0.4192,0,0,1,0,1,2,0,0,1,0,1,2,0,0,0,0,0,0,1,1,5,601,9,14,0,13,9,19,0,13,0,13,8,16,3,7,0,13,47.4000,51.8000,0.0000,45.2000,17.0000,35.0000,0.0000,45.2000,0.0000,45.2000,15.1000,32.1000,5.7000,12.1000,0.0000,45.2000
1,36,New York,NY,10001,ZCTA5 10001,0.6238,27004,1827,16975,831,14375,782,5248,797,761,266,3314,531,1930,534,831,289,3428,432,2694,643,2310,499,501,215,1381,405,13460,2305,15840,898,15,23,389,135,12285,840,2213,218,20.3000,2.7000,4.3000,1.5000,23.1000,3.5000,9.1000,2.4000,3.1000,1.0000,12.7000,1.6000,10.0000,2.1000,8.6000,1.9000,3.5000,1.5000,5.3000,1.5000,49.8000,7.8000,93.3000,2.7000,0.1000,0.1000,2.7000,0.9000,85.5000,2.8000,8.2000,0.6000,0.6108,0.4574,0.5573,0.5902,0.4436,2.6593,0.5688,0.1420,0.1161,0.1891,0.4707,0.8777,1.7956,0.1692,0.8670,0.8670,0.8670,0.9853,0.2710,0.7402,0.9949,0.9104,3.9018,0.9806,9.2237,0.7414,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,3,3,239407,1047,389,2220,576,5206,943,5031,774,0,25,0,25,780,326,223,169,7.3000,2.6000,8.2000,2.2000,19.3000,3.0000,18.6000,2.9000,0.0000,0.1000,0.0000,0.1000,2.9000,1.2000,0.8000,0.6000


In [19]:
def is_nyc_zipcode(zipcode):
    zip_int = int(zipcode) if isinstance(zipcode, str) else zipcode

    # Manhattan: 10001-10282
    if 10001 <= zip_int <= 10282:
        return True
    # addition Manhattan: 10300-10499
    if 10300 <= zip_int <= 10499:
        return True
    # Bronx: 10451-10475
    if 10451 <= zip_int <= 10475:
        return True
    # Brooklyn: 11201-11256
    if 11201 <= zip_int <= 11256:
        return True
    # Queens: 11351-11436, 11101-11109
    if (11351 <= zip_int <= 11436) or (11101 <= zip_int <= 11109):
        return True
    # Staten Island: 10301-10314
    if 10301 <= zip_int <= 10314:
        return True
    # additional Queens ZIPs
    if zip_int in [11004, 11005, 11411, 11412, 11413, 11418, 11419, 11420, 11421, 11422, 11423, 11426, 11427, 11428, 11429]:
        return True
    return False

In [20]:
nyc_df = svi_raw[svi_raw['FIPS'].apply(is_nyc_zipcode)]

In [21]:
nyc_df.shape

(204, 156)

In [22]:
nyc_df.EP_NHPI.unique()
# interesting, so there was an error in the source data

array([ 0.00e+00,  1.00e-01,  3.00e-01,  2.00e-01, -9.99e+02,  8.00e-01,
        1.20e+00,  5.00e-01,  4.00e-01])

In [23]:
svi_raw.EP_NHPI.unique()

array([ 1.51e+01,  0.00e+00,  1.00e-01,  3.00e-01,  2.00e-01, -9.99e+02,
        8.00e-01,  1.20e+00,  5.00e-01,  4.00e-01,  1.10e+00,  7.00e-01,
        1.50e+00,  1.80e+00,  9.00e-01,  6.00e-01,  1.40e+00,  2.10e+00,
        2.30e+00,  1.00e+00])

In [24]:
-9.99e+02, 0.00e+00, 9.00e-01, 2.00e-01

(-999.0, 0.0, 0.9, 0.2)

# **Step 3: All boroughs and their eviction rates**

In [25]:
evictions_pre_post_mean = evictions_pre_post[['ep_afam', 'ep_asian', 'ep_hisp', 'ep_nhpi', 'ep_white', 'ep_twomore', 'ep_otherrace']].mean()
evictions_pre_post_mean
# may need to merge ep_twomore and ep_otherrance toegther

,0
ep_afam,29.2346
ep_asian,8.9040
ep_hisp,38.7307
ep_nhpi,0.0115
ep_white,19.0236
ep_twomore,2.8025
ep_otherrace,1.0239


In [26]:
evictions_pre_post_mean = evictions_pre_post_mean.reset_index()

In [27]:
evictions_pre_post_mean.rename(columns = {'index':'race_svi', 0: "racial percentage"}, inplace=True)

In [28]:
evictions_pre_post_mean

,race_svi,racial percentage
0,ep_afam,29.2346
1,ep_asian,8.9040
2,ep_hisp,38.7307
3,ep_nhpi,0.0115
4,ep_white,19.0236
5,ep_twomore,2.8025
6,ep_otherrace,1.0239


In [29]:
# type(evictions_pre_post_mean)
# so this is correct

In [30]:
# evictions_pre_post.columns

In [160]:
neighbor_evictions = evictions_pre_post.groupby('nta').agg({'average_year_eviction_count': 'mean', 'borough': 'first'}).reset_index()
neighbor_evictions.sort_values('average_year_eviction_count', ascending=False, inplace=True)
neighbor_evictions

,nta,average_year_eviction_count,borough
185,park-cemetery-etc-Bronx,6.4667,BRONX
143,Seagate-Coney Island,4.3904,BROOKLYN
36,Corona,4.2597,QUEENS
163,University Heights-Morris Heights,3.5051,BRONX
125,Park Slope-Gowanus,2.3725,BROOKLYN
75,Grymes Hill-Clifton-Fox Hills,2.3360,STATEN ISLAND
106,Morrisania-Melrose,2.2550,BRONX
19,Bronxdale,2.2396,BRONX
21,Brownsville,2.0909,BROOKLYN
31,Claremont-Bathgate,1.9783,BRONX


In [163]:
man_nta = {
    'nta': neighbor_evictions['nta'].unique(),
    'eviction_rates': neighbor_evictions['average_year_eviction_count']
}

man_nta_df = pd.DataFrame(man_nta)
man_nta_df

,nta,eviction_rates
185,park-cemetery-etc-Bronx,6.4667
143,Seagate-Coney Island,4.3904
36,Corona,4.2597
163,University Heights-Morris Heights,3.5051
125,Park Slope-Gowanus,2.3725
75,Grymes Hill-Clifton-Fox Hills,2.3360
106,Morrisania-Melrose,2.2550
19,Bronxdale,2.2396
21,Brownsville,2.0909
31,Claremont-Bathgate,1.9783


In [164]:
avg_per_nta = neighbor_evictions.average_year_eviction_count.mean()
avg_per_nta
# per building, per year, that's why it is similar to the borough one too

np.float64(0.9049163589883761)

### **Step 3.2. All neighborhoods racial composite**

In [170]:
race_columns = ['ep_afam', 'ep_asian', 'ep_hisp', 'ep_nhpi', 'ep_white', 'ep_twomore', 'ep_otherrace']
racial_avg_all = evictions_pre_post.groupby('nta')[['average_year_eviction_count', 'ep_afam', 'ep_asian', 'ep_hisp', 'ep_nhpi', 'ep_white', 'ep_twomore', 'ep_otherrace', 'rpl_themes']].mean()
racial_avg_all.sort_values('average_year_eviction_count', ascending=False, inplace=True)
racial_avg_all.reset_index(inplace=True)
racial_avg_all

,nta,average_year_eviction_count,ep_afam,ep_asian,ep_hisp,ep_nhpi,ep_white,ep_twomore,ep_otherrace,rpl_themes
0,park-cemetery-etc-Bronx,6.4667,28.6000,6.2000,52.9000,0.0000,8.7000,2.3000,0.7000,0.9925
1,Seagate-Coney Island,4.3904,20.1385,9.8270,19.4159,0.0000,47.3927,2.9144,0.0159,0.9911
2,Corona,4.2597,7.3968,11.3050,75.7668,0.0000,4.4275,0.5039,0.4997,0.9632
3,University Heights-Morris Heights,3.5051,24.4407,1.6079,69.3353,0.0000,2.0544,1.8489,0.7046,0.9979
4,Park Slope-Gowanus,2.3725,7.4853,9.5010,16.3461,0.0000,60.6373,5.4206,0.5765,0.6327
5,Grymes Hill-Clifton-Fox Hills,2.3360,25.1149,11.6023,22.3858,0.0000,38.7568,1.5584,0.4703,0.9444
6,Morrisania-Melrose,2.2550,35.4504,0.6731,59.1572,0.0000,2.5927,1.5814,0.3022,0.9965
7,Bronxdale,2.2396,29.4301,6.1401,52.1306,0.0000,8.6984,2.2915,0.7095,0.9909
8,Brownsville,2.0909,70.0757,0.8424,19.3509,0.0002,3.6573,5.3733,0.6034,0.9933
9,Claremont-Bathgate,1.9783,30.5182,1.0987,63.2627,0.0000,3.0336,1.4791,0.4059,0.9971


In [166]:
racial_avg_all.T

nta,park-cemetery-etc-Bronx,Seagate-Coney Island,Corona,University Heights-Morris Heights,Park Slope-Gowanus,Grymes Hill-Clifton-Fox Hills,Morrisania-Melrose,Bronxdale,Brownsville,Claremont-Bathgate,Westchester-Unionport,Battery Park City-Lower Manhattan,Fordham South,Turtle Bay-East Midtown,Oakwood-Oakwood Beach,Pelham Parkway,East Concourse-Concourse Village,Mount Hope,Ft. Totten-Bay Terrace-Clearview,Melrose South-Mott Haven North,East Tremont,Flushing,Woodlawn-Wakefield,Highbridge,Kew Gardens,Crown Heights South,West Concourse,Central Harlem North-Polo Grounds,Springfield Gardens North,Co-op City,Belmont,Kew Gardens Hills,Midtown-Midtown South,Crotona Park East,Clinton Hill,Bedford Park-Fordham North,Prospect Lefferts Gardens-Wingate,Hunts Point,North Riverdale-Fieldston-Riverdale,Van Cortlandt Village,West Farms-Bronx River,Soundview-Bruckner,Williamsbridge-Olinville,Soundview-Castle Hill-Clason Point-Harding Park,Norwood,West Brighton,Longwood,Dyker Heights,Briarwood-Jamaica Hills,Starrett City,Ocean Hill,Spuyten Duyvil-Kingsbridge,Douglas Manor-Douglaston-Little Neck,Murray Hill,Kingsbridge Heights,Fort Greene,Parkchester,Mariner's Harbor-Arlington-Port Ivory-Graniteville,East Harlem North,East Williamsburg,East Flatbush-Farragut,Jamaica,Hudson Yards-Chelsea-Flatiron-Union Square,Bushwick South,DUMBO-Vinegar Hill-Downtown Brooklyn-Boerum Hill,Rego Park,East New York,East Harlem South,Marble Hill-Inwood,Sheepshead Bay-Gerritsen Beach-Manhattan Beach,North Side-South Side,Midwood,Auburndale,Mott Haven-Port Morris,Forest Hills,Hollis,Erasmus,Bushwick North,Bedford,Lincoln Square,Crown Heights North,Elmhurst,Brighton Beach,Flatlands,Hunters Point-Sunnyside-West Maspeth,Fresh Meadows-Utopia,Rugby-Remsen Village,Washington Heights South,Flatbush,Lower East Side,Upper West Side,Clinton,Washington Heights North,West New Brighton-New Brighton-St. George,Elmhurst-Maspeth,Bensonhurst East,Jamaica Estates-Holliswood,Jackson Heights,Williamsburg,Homecrest,Central Harlem South,Hamilton Heights,Carroll Gardens-Columbia Street-Red Hook,Pelham Bay-Country Club-City Island,Bayside-Bayside Hills,Stuyvesant Heights,Canarsie,East New York (Pennsylvania Ave),Manhattanville,Ocean Parkway South,Stapleton-Rosebank,Madison,Bath Beach,Steinway,Murray Hill-Kips Bay,Chinatown,Allerton-Pelham Gardens,Gravesend,Queens Village,Upper East Side-Carnegie Hill,Cypress Hills-City Line,Yorkville,Bensonhurst West,Van Nest-Morris Park-Westchester Square,Bay Ridge,Stuyvesant Town-Cooper Village,Morningside Heights,SoHo-TriBeCa-Civic Center-Little Italy,Eastchester-Edenwald-Baychester,New Brighton-Silver Lake,Gramercy,Woodhaven,Old Astoria,South Jamaica,Kensington-Ocean Parkway,Grasmere-Arrochar-Ft. Wadsworth,New Dorp-Midland Beach,Rosedale,Woodside,St. Albans,Astoria,Lenox Hill-Roosevelt Island,Glendale,Todt Hill-Emerson Hill-Heartland Village-Lighthouse Hill,Schuylerville-Throgs Neck-Edgewater Park,Laurelton,Old Town-Dongan Hills-South Beach,Ridgewood,Pomonok-Flushing Heights-Hillcrest,Middle Village,Baisley Park,East Elmhurst,Richmond Hill,Queensbridge-Ravenswood-Long Island City,Georgetown-Marine Park-Bergen Beach-Mill Basin,Sunset Park East,Springfield Gardens South-Brookville,South Ozone Park,College Point,Sunset Park West,East Flushing,Port Richmond,Brooklyn Heights-Cobble Hill,Borough Park,Prospect Heights,Windsor Terrace,North Corona,Whitestone,Ozone Park,Lindenwood-Howard Beach,West Village,Cambria Heights,East Village,New Springville-Bloomfield-Travis,Bellerose,Queensboro Hill,Oakland Gardens,Greenpoint,Charleston-Richmond Valley-Tottenville,Great Kills,Annadale-Huguenot-Prince's Bay-Eltingville,Westerleigh,Maspeth,Arden Heights,Glen Oaks-Floral Park-New Hyde Park,Rossville-Woodrow,park-cemetery-etc-Brooklyn
average_year_eviction_count,6.4667,4.3904,4.2597,3.5051,2.3725,2.3360,2.2550,2.2396,2.0909,1.9783,1.9408,1.7746,1.6845,1.6833,1.6722,1.6411,1.6058,1.5958,1.5800,1.5741,1.5485,1.5223,1.5069,1.4761,1.4738,1.4379,1.4167,1.4098,1.4075,1.4012,1.3974,1

## **This is for the race composites and neighborhoods bar char use**

# **Step 4 We want a dataframe that has neighborhoods as columns and average_year_eviction_count as rows and contents**

In [171]:
evi_svi_df = evictions_pre_post.groupby('nta')[['average_year_eviction_count','rpl_themes']].mean()
evi_svi_df.sort_values('average_year_eviction_count', ascending=False, inplace=True)
evi_svi_df

,average_year_eviction_count,rpl_themes
nta,,
park-cemetery-etc-Bronx,6.4667,0.9925
Seagate-Coney Island,4.3904,0.9911
Corona,4.2597,0.9632
University Heights-Morris Heights,3.5051,0.9979
Park Slope-Gowanus,2.3725,0.6327
Grymes Hill-Clifton-Fox Hills,2.3360,0.9444
Morrisania-Melrose,2.2550,0.9965
Bronxdale,2.2396,0.9909
Brownsville,2.0909,0.9933


In [145]:
evi_svi_df.reset_index(inplace=True)
evi_svi_df

,nta,average_year_eviction_count,rpl_themes
0,park-cemetery-etc-Bronx,6.4667,0.9925
1,Seagate-Coney Island,4.3904,0.9911
2,Corona,4.2597,0.9632
3,University Heights-Morris Heights,3.5051,0.9979
4,Park Slope-Gowanus,2.3725,0.6327
5,Grymes Hill-Clifton-Fox Hills,2.3360,0.9444
6,Morrisania-Melrose,2.2550,0.9965
7,Bronxdale,2.2396,0.9909
8,Brownsville,2.0909,0.9933
9,Claremont-Bathgate,1.9783,0.9971


In [146]:
avg_per_nta

np.float64(0.9049163589883761)

In [147]:
average_svi = 0.80198
# https://www.atsdr.cdc.gov/place-health/php/svi/svi-interactive-map.html

In [202]:
evi_svi_df['above_eviction_average'] = evi_svi_df['average_year_eviction_count'] > avg_per_nta
evi_svi_df

,average_year_eviction_count,rpl_themes,above_svi_average,above_eviction_average
nta,,,,
park-cemetery-etc-Bronx,6.4667,0.9925,True,True
Seagate-Coney Island,4.3904,0.9911,True,True
Corona,4.2597,0.9632,True,True
University Heights-Morris Heights,3.5051,0.9979,True,True
Park Slope-Gowanus,2.3725,0.6327,False,True
Grymes Hill-Clifton-Fox Hills,2.3360,0.9444,True,True
Morrisania-Melrose,2.2550,0.9965,True,True
Bronxdale,2.2396,0.9909,True,True
Brownsville,2.0909,0.9933,True,True


In [203]:
evi_svi_df['above_svi_average'] = evi_svi_df['rpl_themes'] > average_svi
evi_svi_df

,average_year_eviction_count,rpl_themes,above_svi_average,above_eviction_average
nta,,,,
park-cemetery-etc-Bronx,6.4667,0.9925,True,True
Seagate-Coney Island,4.3904,0.9911,True,True
Corona,4.2597,0.9632,True,True
University Heights-Morris Heights,3.5051,0.9979,True,True
Park Slope-Gowanus,2.3725,0.6327,False,True
Grymes Hill-Clifton-Fox Hills,2.3360,0.9444,True,True
Morrisania-Melrose,2.2550,0.9965,True,True
Bronxdale,2.2396,0.9909,True,True
Brownsville,2.0909,0.9933,True,True


# **Step 4.3 Run Chi-Square test**
  **Null Hypothesis: There is no association between neighborhoods that have above the average of black and hispanic percentage and have above than average svi scores**

In [204]:
contingency_table = pd.crosstab(evi_svi_df.above_svi_average, evi_svi_df.above_eviction_average)
contingency_table

above_eviction_average,False,True
above_svi_average,,
False,29,9
True,86,63


In [205]:
chi2, p_value, dof, expected = chi2_contingency(contingency_table)
chi2, p_value, dof, expected

(np.float64(3.672242587459067),
 np.float64(0.05532569618900214),
 1,
 array([[23.36898396, 14.63101604],
        [91.63101604, 57.36898396]]))

### **P-value is 0.05533 and chi-square is 3.67. We can not refute the null hypothesis. However the p-value is at borderline and the chi-square suggests a small-to-moderate deviation from independence, and therefore it is worthwile to look into it.**

# **conclusion: there is no statistically significant association between the two "above average eviction" and "above average svi scores" neighborhoods.**

In [153]:
a = evi_svi_df[(evi_svi_df['above_svi_average'] == True) & (evi_svi_df['above_eviction_average'] == True)].shape[0]
b = evi_svi_df[(evi_svi_df['above_svi_average'] == True) & (evi_svi_df['above_eviction_average'] == False)].shape[0]
c = evi_svi_df[(evi_svi_df['above_svi_average'] == False) & (evi_svi_df['above_eviction_average'] == True)].shape[0]
d = evi_svi_df[(evi_svi_df['above_svi_average'] == False) & (evi_svi_df['above_eviction_average'] == False)].shape[0]

In [156]:
observed = np.array([[a, b], [c, d]])
observed

array([[63.5, 86.5],
       [ 9.5, 29.5]])

In [157]:
# a += 0.5
# b += 0.5
# c += 0.5
# d += 0.5
# # avoid 0 divisions

In [158]:
odds_ratio = (a * d) / (b * c)
odds_ratio
# extremely strong association between high svi and high evitction rates.

2.2795862488591423

**This means, technically, neighborhoods with above average svi had 2.28 times higher odds of having above average eviction rates compared to low-SVI neighborhoods.** This high odds may be due to all high svi neighborhoods had high evictions, and all high eviction neighborhoods had high svi from the sample.

In [159]:
chi2, p_value, dof, expected = chi2_contingency(observed)
chi2, p_value

(np.float64(3.494209694415174), np.float64(0.061583795661582674))

# **Step 5: We also need a dataframe that has neighborhoods as columns and black and hispanics percentages as rows and contents**

In [173]:
racial_avg_all['black_hispanic_pct'] = racial_avg_all['ep_afam'] + racial_avg_all['ep_hisp']
racial_avg_all

,nta,average_year_eviction_count,ep_afam,ep_asian,ep_hisp,ep_nhpi,ep_white,ep_twomore,ep_otherrace,rpl_themes,black_hispanic_pct
0,park-cemetery-etc-Bronx,6.4667,28.6000,6.2000,52.9000,0.0000,8.7000,2.3000,0.7000,0.9925,81.5000
1,Seagate-Coney Island,4.3904,20.1385,9.8270,19.4159,0.0000,47.3927,2.9144,0.0159,0.9911,39.5544
2,Corona,4.2597,7.3968,11.3050,75.7668,0.0000,4.4275,0.5039,0.4997,0.9632,83.1637
3,University Heights-Morris Heights,3.5051,24.4407,1.6079,69.3353,0.0000,2.0544,1.8489,0.7046,0.9979,93.7760
4,Park Slope-Gowanus,2.3725,7.4853,9.5010,16.3461,0.0000,60.6373,5.4206,0.5765,0.6327,23.8314
5,Grymes Hill-Clifton-Fox Hills,2.3360,25.1149,11.6023,22.3858,0.0000,38.7568,1.5584,0.4703,0.9444,47.5007
6,Morrisania-Melrose,2.2550,35.4504,0.6731,59.1572,0.0000,2.5927,1.5814,0.3022,0.9965,94.6076
7,Bronxdale,2.2396,29.4301,6.1401,52.1306,0.0000,8.6984,2.2915,0.7095,0.9909,81.5607
8,Brownsville,2.0909,70.0757,0.8424,19.3509,0.0002,3.6573,5.3733,0.6034,0.9933,89.4267
9,Claremont-Bathgate,1.9783,30.5182,1.0987,63.2627,0.0000,3.0336,1.4791,0.4059,0.9971,93.7809


In [180]:
evi_bh_df = racial_avg_all[['nta', 'average_year_eviction_count', 'black_hispanic_pct']]
evi_bh_df

,nta,average_year_eviction_count,black_hispanic_pct
0,park-cemetery-etc-Bronx,6.4667,81.5000
1,Seagate-Coney Island,4.3904,39.5544
2,Corona,4.2597,83.1637
3,University Heights-Morris Heights,3.5051,93.7760
4,Park Slope-Gowanus,2.3725,23.8314
5,Grymes Hill-Clifton-Fox Hills,2.3360,47.5007
6,Morrisania-Melrose,2.2550,94.6076
7,Bronxdale,2.2396,81.5607
8,Brownsville,2.0909,89.4267
9,Claremont-Bathgate,1.9783,93.7809


In [181]:
average_bh_pct = evi_bh_df['black_hispanic_pct'].mean()
average_bh_pct

np.float64(49.541793265928135)

In [182]:
avg_per_nta, average_bh_pct
# evictions and black + hispanic pct

(np.float64(0.9049163589883761), np.float64(49.541793265928135))

In [183]:
evi_bh_df['above_evi_average'] = evi_bh_df['average_year_eviction_count'] > avg_per_nta
evi_bh_df

,nta,average_year_eviction_count,black_hispanic_pct,above_evi_average
0,park-cemetery-etc-Bronx,6.4667,81.5000,True
1,Seagate-Coney Island,4.3904,39.5544,True
2,Corona,4.2597,83.1637,True
3,University Heights-Morris Heights,3.5051,93.7760,True
4,Park Slope-Gowanus,2.3725,23.8314,True
5,Grymes Hill-Clifton-Fox Hills,2.3360,47.5007,True
6,Morrisania-Melrose,2.2550,94.6076,True
7,Bronxdale,2.2396,81.5607,True
8,Brownsville,2.0909,89.4267,True
9,Claremont-Bathgate,1.9783,93.7809,True


In [187]:
evi_bh_df['above_bh_average'] = evi_bh_df['black_hispanic_pct'] > average_bh_pct
evi_bh_df

,nta,average_year_eviction_count,black_hispanic_pct,above_evi_average,above_bh_average
0,park-cemetery-etc-Bronx,6.4667,81.5000,True,True
1,Seagate-Coney Island,4.3904,39.5544,True,False
2,Corona,4.2597,83.1637,True,True
3,University Heights-Morris Heights,3.5051,93.7760,True,True
4,Park Slope-Gowanus,2.3725,23.8314,True,False
5,Grymes Hill-Clifton-Fox Hills,2.3360,47.5007,True,False
6,Morrisania-Melrose,2.2550,94.6076,True,True
7,Bronxdale,2.2396,81.5607,True,True
8,Brownsville,2.0909,89.4267,True,True
9,Claremont-Bathgate,1.9783,93.7809,True,True


### **This is the neighborhood evictions + black/hispanic chi-test df:**

# **Run Chi-Square test**
  **Null Hypothesis: There is no association between neighborhoods that have above the average of black and hispanic percentage and have above than average svi scores**

In [188]:
contingency_table = pd.crosstab(evi_bh_df.above_bh_average, evi_bh_df.above_evi_average)
contingency_table

above_evi_average,False,True
above_bh_average,,
False,80,27
True,35,45


In [189]:
chi2, p_value, dof, expected = chi2_contingency(contingency_table)
chi2, p_value, dof, expected

(np.float64(17.311162196233465),
 np.float64(3.1731742700266107e-05),
 1,
 array([[65.80213904, 41.19786096],
        [49.19786096, 30.80213904]]))

### **P-value is 0.00003.17 (extremely small) and chi-squiare is 17.311. We can refute the null hypothesis. We can say we are confident that high "black+hispanic percetage" boroughs are more likely to have high evictions (or vice versa) or there is a strong association between high BH pct and high eviction rates in the full dataset**

# **conclusion: there is a statistically significant association between the two "above average eviction" and "above average black+hispanic pct" neighborhoods.**

In [190]:
a = evi_bh_df[(evi_bh_df['above_bh_average'] == True) & (evi_bh_df['above_evi_average'] == True)].shape[0]
b = evi_bh_df[(evi_bh_df['above_bh_average'] == True) & (evi_bh_df['above_evi_average'] == False)].shape[0]
c = evi_bh_df[(evi_bh_df['above_bh_average'] == False) & (evi_bh_df['above_evi_average'] == True)].shape[0]
d = evi_bh_df[(evi_bh_df['above_bh_average'] == False) & (evi_bh_df['above_evi_average'] == False)].shape[0]

In [191]:
observed = np.array([[a, b], [c, d]])
observed

array([[45, 35],
       [27, 80]])

In [193]:
odds_ratio = (a * d) / (b * c)
odds_ratio

3.8095238095238093

**This means, technically, neighborhoods with above average b+h pct had 3.81 times higher odds of having above average eviction rates compared to low-bh-pct neighborhoods in nyc.** This high odds may be due to all high bh pct neighborhoods had high evictions, and all high eviction neighborhoods had high bh pct from the sample.

In [194]:
chi2, p_value, dof, expected = chi2_contingency(observed)
chi2, p_value
# 11.32 is a strong deviation from expected undert the null hypothesis

(np.float64(17.311162196233465), np.float64(3.1731742700266107e-05))

### **Quick extreme cases**

In [207]:
evi_svi_df_top_10 = evi_svi_df.sort_values('average_year_eviction_count', ascending=False).head(10)
evi_svi_df_bottom_10 = evi_svi_df.sort_values('average_year_eviction_count', ascending=False).tail(10)
evi_svi_df_extremes = pd.concat([evi_svi_df_top_10, evi_svi_df_bottom_10])
evi_svi_df_extremes

,average_year_eviction_count,rpl_themes,above_svi_average,above_eviction_average
nta,,,,
park-cemetery-etc-Bronx,6.4667,0.9925,True,True
Seagate-Coney Island,4.3904,0.9911,True,True
Corona,4.2597,0.9632,True,True
University Heights-Morris Heights,3.5051,0.9979,True,True
Park Slope-Gowanus,2.3725,0.6327,False,True
Grymes Hill-Clifton-Fox Hills,2.3360,0.9444,True,True
Morrisania-Melrose,2.2550,0.9965,True,True
Bronxdale,2.2396,0.9909,True,True
Brownsville,2.0909,0.9933,True,True


In [211]:
contingency_table = pd.crosstab(evi_svi_df_extremes.above_svi_average, evi_svi_df_extremes.above_eviction_average)
contingency_table

above_eviction_average,False,True
above_svi_average,,
False,7,1
True,3,9


In [212]:
chi2, p_value, dof, expected = chi2_contingency(contingency_table)
chi2, p_value, dof, expected

(np.float64(5.208333333333334),
 np.float64(0.022478873366125265),
 1,
 array([[4., 4.],
        [6., 6.]]))

### **P-value is 0.022 (extremely small) and chi-squiare is 5.208. We can refute the null hypothesis. We can say we are confident that high "svi" boroughs are more likely to have high evictions (or vice versa) or there is a strong association between high SVI and high eviction rates in the full dataset**

# **conclusion: there is a statistically significant association between the two "above average eviction" and "above average black+hispanic pct" neighborhoods.**

In [214]:
a = evi_svi_df_extremes[(evi_svi_df_extremes['above_svi_average'] == True) & (evi_svi_df_extremes['above_eviction_average'] == True)].shape[0]
b = evi_svi_df_extremes[(evi_svi_df_extremes['above_svi_average'] == True) & (evi_svi_df_extremes['above_eviction_average'] == False)].shape[0]
c = evi_svi_df_extremes[(evi_svi_df_extremes['above_svi_average'] == False) & (evi_svi_df_extremes['above_eviction_average'] == True)].shape[0]
d = evi_svi_df_extremes[(evi_svi_df_extremes['above_svi_average'] == False) & (evi_svi_df_extremes['above_eviction_average'] == False)].shape[0]

In [215]:
observed = np.array([[a, b], [c, d]])
observed

array([[9, 3],
       [1, 7]])

In [216]:
odds_ratio = (a * d) / (b * c)
odds_ratio

21.0

**This means, technically, neighborhoods with above average b+h pct had 21 times higher odds of having above average eviction rates compared to low-bh-pct neighborhoods in nyc.** This high odds may be due to all high bh pct neighborhoods had high evictions, and all high eviction neighborhoods had high bh pct from the sample.

In [217]:
evi_bh_df_top_10 = evi_bh_df.sort_values('average_year_eviction_count', ascending=False).head(10)
evi_bh_df_bottom_10 = evi_bh_df.sort_values('average_year_eviction_count', ascending=False).tail(10)
evi_bh_df_extremes = pd.concat([evi_bh_df_top_10, evi_bh_df_bottom_10])
evi_bh_df_extremes

,nta,average_year_eviction_count,black_hispanic_pct,above_evi_average,above_bh_average
0,park-cemetery-etc-Bronx,6.4667,81.5000,True,True
1,Seagate-Coney Island,4.3904,39.5544,True,False
2,Corona,4.2597,83.1637,True,True
3,University Heights-Morris Heights,3.5051,93.7760,True,True
4,Park Slope-Gowanus,2.3725,23.8314,True,False
5,Grymes Hill-Clifton-Fox Hills,2.3360,47.5007,True,False
6,Morrisania-Melrose,2.2550,94.6076,True,True
7,Bronxdale,2.2396,81.5607,True,True
8,Brownsville,2.0909,89.4267,True,True
9,Claremont-Bathgate,1.9783,93.7809,True,True


In [220]:
contingency_table = pd.crosstab(evi_bh_df_extremes.above_bh_average, evi_bh_df_extremes.above_evi_average)
contingency_table

above_evi_average,False,True
above_bh_average,,
False,10,3
True,0,7


In [221]:
chi2, p_value, dof, expected = chi2_contingency(contingency_table)
chi2, p_value, dof, expected

(np.float64(7.912087912087912),
 np.float64(0.004910556162172649),
 1,
 array([[6.5, 6.5],
        [3.5, 3.5]]))

### **P-value is 0.0049 (extremely small) and chi-squiare is 7.812. We can refute the null hypothesis. We can say we are confident that high "black+hispanic" boroughs are more likely to have high evictions (or vice versa) or there is a strong association between high black+hispanic and high eviction rates in the full dataset**

# **conclusion: there is a statistically significant association between the two "above average eviction" and "above average black+hispanic pct" neighborhoods.**

In [224]:
a = evi_bh_df_extremes[(evi_bh_df_extremes['above_bh_average'] == True) & (evi_bh_df_extremes['above_evi_average'] == True)].shape[0]
b = evi_bh_df_extremes[(evi_bh_df_extremes['above_bh_average'] == True) & (evi_bh_df_extremes['above_evi_average'] == False)].shape[0]
c = evi_bh_df_extremes[(evi_bh_df_extremes['above_bh_average'] == False) & (evi_bh_df_extremes['above_evi_average'] == True)].shape[0]
d = evi_bh_df_extremes[(evi_bh_df_extremes['above_bh_average'] == False) & (evi_bh_df_extremes['above_evi_average'] == False)].shape[0]

In [225]:
observed = np.array([[a, b], [c, d]])
observed

array([[ 7,  0],
       [ 3, 10]])

In [226]:
a += 0.5
b += 0.5
c += 0.5
d += 0.5

In [227]:
odds_ratio = (a * d) / (b * c)
odds_ratio

45.0

**This means, technically, neighborhoods with above average b+h pct had 45 times higher odds of having above average eviction rates compared to low-bh-pct neighborhoods in nyc.** This high odds may be due to all high bh pct neighborhoods had high evictions, and all high eviction neighborhoods had high bh pct from the sample.